# Day 9 solution using a class

Refactoring solutions for Day 9 to use and extension of Day 7 class, in case this is needed in the future

In [1]:
import numpy as np

In [2]:
def getDigits(num):
    digits = []
    while num > 0:
        d = num%10
        num = num//10
        digits.append(d)
    return digits

def repackInstr(op):
    dig = getDigits(op)
    instr = [0,0,0,0]
    n = 0
    for i in dig:
        if (n==0): instr[0] = i
        elif (n==1): instr[0] += 10*i
        else: instr[n-1] = i
        n += 1
    return instr

op = 21107
print(repackInstr(op))

[7, 1, 1, 2]


In [3]:
class amplifier():
    """Amplifier"""

    def __init__(self, prog, phase=0, name="AMP", debug=False):
        # Increase memory size to account for Day 9 programs
        memsize = 1000000
        prog_ = np.array(prog)
        self.digits = np.zeros((memsize,), dtype=int)
        self.digits[:len(prog_)] = prog_
        #
        self.debug = debug
        self.name = name
        self.phase = phase
        self.relbase = 0 # need for OpInt 9
        self.ip = 0
        self.lastOutput = -1
        self.output = []
        self.iinp = 0
        self.isRunning = True
        self.isHalted = False
        self.step = 0

    def getLastOutput(self):
        return self.lastOutput
        
    def getOutput(self):
        return self.output
    
    def getStep(self):
        return self.step
    
    def runProgram(self,theInput=0):
        if self.isHalted:
            self.isRunning = False
            return
        self.isRunning = True
        inputs = [self.phase,theInput]
        outputDig = 0
        if (self.ip and self.isRunning and self.debug):
            print("Resuming",self.name,"operation at instruction",self.ip,"with input",theInput)
        while ( self.ip < len(self.digits) and self.isRunning ):
            self.step +=1
            op_ = self.digits[self.ip]
            if op_ == 99: 
                if (self.debug):
                    print(self.name,"halted - Final output =",self.output)
                self.isRunning = False
                self.isHalted = True
                self.isRunning
                break
            p = repackInstr(op_)
            op = p[0]
            if op == 3:
                inputDig = inputs[self.iinp]
                self.iinp=1
                a = self.digits[self.ip+1]
                if   p[1]==0: self.digits[a] = inputDig
                elif p[1]==1: self.digits[self.digits[a]] = inputDig
                elif p[1]==2: self.digits[a+self.relbase] = inputDig
                self.ip += 2
            elif op==4:
                a = self.digits[self.ip+1]
                if   p[1]==0: outputDig = self.digits[a]
                elif p[1]==1: outputDig = a
                elif p[1]==2: outputDig = self.digits[a+self.relbase]
                self.ip += 2
                self.lastOutput = outputDig
                self.output.append(outputDig)
                self.isRunning = False
                if (self.debug):
                    print(self.name,"is waiting for new input at instruction",self.ip,"- Current output =",self.lastOutput)
            elif op==5:
                a, b = self.digits[self.ip+1:self.ip+3]
                if   p[1]==0: a_ = self.digits[a]
                elif p[1]==1: a_ = a
                elif p[1]==2: a_ = self.digits[a+self.relbase]
                if   p[2]==0: b_ = self.digits[b]
                elif p[2]==1: b_ = b
                elif p[2]==2: b_ = self.digits[b+self.relbase]
                if a_: self.ip = b_
                else: self.ip += 3
            elif op==6:
                a, b = self.digits[self.ip+1:self.ip+3]
                if   p[1]==0: a_ = self.digits[a]               
                elif p[1]==1: a_ = a
                elif p[1]==2: a_ = self.digits[a+self.relbase]
                if   p[2]==0: b_ = self.digits[b]
                elif p[2]==1: b_ = b
                elif p[2]==2: b_ = self.digits[b+self.relbase]
                if not a_: self.ip = b_
                else: self.ip += 3
            elif op==7:
                a, b, c = self.digits[self.ip+1:self.ip+4]
                if   p[1]==0: a_ = self.digits[a]
                elif p[1]==1: a_ = a
                elif p[1]==2: a_ = self.digits[a+self.relbase]
                if   p[2]==0: b_ = self.digits[b]
                elif p[2]==1: b_ = b
                elif p[2]==2: b_ = self.digits[b+self.relbase]
                if   p[3]==0: c_ = c
                elif p[3]==1: c_ = self.digits[c]
                elif p[3]==2: c_ = c+self.relbase
                if a_ < b_: self.digits[c_] = 1
                else: self.digits[c_] = 0
                self.ip += 4
            elif op==8:
                a, b, c = self.digits[self.ip+1:self.ip+4]
                if   p[1]==0: a_ = self.digits[a]
                elif p[1]==1: a_ = a
                elif p[1]==2: a_ = self.digits[a+self.relbase]    
                if   p[2]==0: b_ = self.digits[b]
                elif p[2]==1: b_ = b
                elif p[2]==2: b_ = self.digits[b+self.relbase]
                if   p[3]==0: c_ = c
                elif p[3]==1: c_ = self.digits[c]
                elif p[3]==2: c_ = c+self.relbase
                if a_ == b_: self.digits[c_] = 1
                else: self.digits[c_] = 0
                self.ip += 4
            elif op==9:
                a = self.digits[self.ip+1]
                if   p[1]==0: self.relbase += self.digits[a]
                elif p[1]==1: self.relbase += a
                elif p[1]==2: self.relbase += self.digits[a+self.relbase] # needed?
                self.ip += 2    
            elif op == 1 or op == 2: 
                a, b, c = self.digits[self.ip+1:self.ip+4]  
                if   p[1]==0: a_ = self.digits[a]
                elif p[1]==1: a_ = a
                elif p[1]==2: a_ = self.digits[a+self.relbase]    
                if   p[2]==0: b_ = self.digits[b] 
                elif p[2]==1: b_ = b   
                elif p[2]==2: b_ = self.digits[b+self.relbase]
                if   p[3]==0: c_ = c
                elif p[3]==1: c_ = self.digits[c]
                elif p[3]==2: c_ = c+self.relbase
                if op == 1: self.digits[c_] = a_ + b_
                if op == 2: self.digits[c_] = a_ * b_
                self.ip += 4
            else:
                if (self.debug):
                    print("Illegal operator:",op)
                self.isRunning = False
                return self.isRunning
        return self.isRunning

## Day 5 emulation

In [4]:
# Emulate Day 5 function with amplifier class

def program5(digits,inp=0,debug=False):
    prog = list(digits)
    #phase = inp # assume no phase is used (yet), thus input and phase are the same
    amp = amplifier(prog) #,phase,"AMP",debug)
    while(not amp.isHalted):
        #amp.runProgram(inp)
        amp.runProgram()
    return amp.getLastOutput(), amp.getStep()

In [5]:
# Consider whether the input is equal to 8; output 1 (if it is) or 0 (if it is not).
#inp = 8
#prog = [3,9,8,9,10,9,4,9,99,-1,8] 

# Consider whether the input is less than 8; output 1 (if it is) or 0 (if it is not).
#inp = 7
#prog = [3,9,7,9,10,9,4,9,99,-1,8] 

# Consider whether the input is equal to 8; output 1 (if it is) or 0 (if it is not).
#inp = 8
#prog = [3,3,1108,-1,8,3,4,3,99]

# Consider whether the input is less than 8; output 1 (if it is) or 0 (if it is not).
inp = 7
prog = [3,3,1107,-1,8,3,4,3,99]

output, steps = program5(prog,inp)
print("outout =",output)
print("steps  =", steps)

outout = 1
steps  = 4


## Day 9 emulation

In [6]:
def program9(digits,inp=0,debug=False):
    prog = list(digits)
    phase = inp # assume no phase is used (yet), thus input and phase are the same
    amp = amplifier(prog,phase,"AMP",debug)
    while(not amp.isHalted):
        amp.runProgram(inp)
    return amp.getOutput(), amp.getStep()

In [7]:
prog = [109,1,204,-1,1001,100,1,100,1008,100,16,101,1006,101,0,99] # takes no input and produces a copy of itself as output
#prog = [ 1102,34915192,34915192,7,4,7,99,0] # should output a 16-digit number.
#prog = [104,1125899906842624,99] # should output the large number in the middle.

output, steps = program9(prog,0)
print("outout =",output)
print("steps  =", steps)

outout = [109, 1, 204, -1, 1001, 100, 1, 100, 1008, 100, 16, 101, 1006, 101, 0, 99]
steps  = 81


In [8]:
def readProg(infile):
    with open(infile) as f:
        lines = [l.rstrip('\n') for l in f]
        return list([int(c) for c in lines[0].split(',')])

In [9]:
fullprog = readProg("./data/input9.txt")

In [10]:
inp = 1
output,steps = program9(fullprog,inp)
print("BOOST signal =",output)
print("Steps        =",steps)

BOOST signal = [2752191671]
Steps        = 206


In [11]:
import time
start_time = time.time()

inp = 2
output, steps = program9(fullprog,inp)

print("BOOST signal   =",output[0])
print("Steps          =",steps)
print("Execution time = %f seconds" % (time.time() - start_time))

BOOST signal   = 87571
Steps          = 371206
Execution time = 4.458852 seconds
